In [1]:
pip install googletrans==4.0.0-rc1

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 16.1 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17412 sha256=7a31af46c839337ef789fd6a8a9e87ebac73d5b7466972abde87fb36bcceee83
  Stored in directory: /home/jovyan/.cache/pip/wheels/60/b3/27/d8aff3e2d5c2d0d97a117cdf0d5f13cd121e2c2b5fb49b55a0
Successfully built googletrans
  Consider addi

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from langchain.tools import tool
from googletrans import Translator

In [3]:
@tool
def translate_text(text: str, target_languages: list) -> dict:
    """
    Translates the input text into specified target languages.

    Args:
        text (str): The text to be translated.
        target_languages (list): List of target languages in ISO 639-1 codes.

    Returns:
        dict: A dictionary with the language full form as keys and their translations as values.
    """
    translator = Translator()
    translations = {}

    for lang_code in target_languages:
        translated = translator.translate(text, dest=lang_code)
        translations[lang_code] = translated.text

    return translations

In [4]:
tools = [translate_text]

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [12]:
translate_text_function =format_tool_to_openai_function(translate_text)
model = ChatOpenAI(temperature=0).bind(functions = [translate_text_function])

In [13]:
from langchain.prompts import MessagesPlaceholder
translation_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "Translate the following text: '{text}' into the following languages: {target_languages}."),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [14]:
chain =translation_prompt | model | OpenAIFunctionsAgentOutputParser()

In [15]:
result1 = chain.invoke({
    "text": "How are you?",
    "target_languages": "fr,hi",
    "agent_scratchpad": [],
    "chat_history": []
})

In [16]:
result1.tool

'translate_text'

In [17]:
observation = translate_text(result1.tool_input)

In [18]:
observation

{'fr': 'Comment vas-tu?', 'hi': 'आप कैसे हैं?'}

In [19]:
type(result1)

langchain.schema.agent.AgentActionMessageLog

In [20]:
from langchain.agents.format_scratchpad import format_to_openai_functions

In [21]:
result1.message_log

[AIMessage(content='', additional_kwargs={'function_call': {'name': 'translate_text', 'arguments': '{"text":"How are you?","target_languages":["fr","hi"]}'}})]

In [22]:
format_to_openai_functions([(result1, observation), ])

[AIMessage(content='', additional_kwargs={'function_call': {'name': 'translate_text', 'arguments': '{"text":"How are you?","target_languages":["fr","hi"]}'}}),
 FunctionMessage(content='{"fr": "Comment vas-tu?", "hi": "आप कैसे हैं?"}', name='translate_text')]

In [23]:
from langchain.schema.runnable import RunnablePassthrough
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | translation_prompt | model | OpenAIFunctionsAgentOutputParser()

In [30]:
from langchain.schema.agent import AgentFinish
def run_agent(user_input, target_languages):
    intermediate_steps = []
    chat_history = [] 
    while True:
        
        result = agent_chain.invoke({
            "text": user_input, 
            "target_languages": target_languages,
            "intermediate_steps": intermediate_steps,
            "chat_history": chat_history
        })

        if isinstance(result, AgentFinish):
            return result.output

        tool = {
            "translate_text": translate_text,
            
        }[result.tool]

        observation = tool(result.tool_input)
        intermediate_steps.append((result, observation))
        return observation

In [31]:
run_agent("Hello", "fr,hi,es,ja")

{'fr': 'Bonjour', 'hi': 'नमस्ते', 'es': 'Hola', 'ja': 'こんにちは'}

In [32]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=False)

In [33]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")

In [34]:
input_data = {
    "text": "What is LangChain?",
    "target_languages": "es,fr,de,hi,ru",
    "chat_history": [] 
    
}

In [41]:
output_translations = agent_executor.invoke(input_data)
output_translations["output"]

"Here is the translation of the text 'What is LangChain?' into the specified languages:\n- Spanish: ¿Qué es LangChain?\n- French: Qu'est-ce que LangChain?\n- German: Was ist LangChain?\n- Hindi: लैंगचेन क्या है?\n- Russian: Что такое LangChain?"

#### CREATE A CHATBOT


In [36]:
@tool
def create_your_own(query: str) -> str:
    """This function can do whatever you would like once you fill it in """
    print(type(query))
    return query[::-1]

In [42]:
import panel as pn  # GUI
pn.extension()
import panel as pn
import param

class cbfs(param.Parameterized):
    
    def __init__(self, tools, **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.functions = [format_tool_to_openai_function(f) for f in tools]
        self.model = ChatOpenAI(temperature=0).bind(functions=self.functions)
        self.memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a friendly translation chatbot. If the input is to translate a text, then use the translate_text provided. If the input is not a translation task then have a friendly conversation with the user. Do not make up any information. Do not tell that your using the transalate_text to perform translations."),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        self.chain = RunnablePassthrough.assign(
            agent_scratchpad = lambda x: format_to_openai_functions(x["intermediate_steps"])
        ) | self.prompt | self.model | OpenAIFunctionsAgentOutputParser()
        self.qa = AgentExecutor(agent=self.chain, tools=tools, verbose=False, memory=self.memory)
    
    def convchain(self, query):
        if not query:
            return
        inp.value = ''
        result = self.qa.invoke({"input": query})
        self.answer = result['output'] 
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=450)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=450, styles={'background-color': '#F6F6F6'}))
        ])
        return pn.WidgetBox(*self.panels, scroll=True)


    def clr_history(self,count=0):
        self.chat_history = []
        return 

In [44]:
cb = cbfs(tools)

inp = pn.widgets.TextInput( placeholder='Enter text here…')

conversation = pn.bind(cb.convchain, inp) 

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=400),
    pn.layout.Divider(),
)

dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# Translation Chatbot')),
    pn.Tabs(('Conversation', tab1))
)
dashboard

Column
    [0] Row
        [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Row
                [0] TextInput(placeholder='Enter text here…')
            [1] Divider()
            [2] ParamFunction(function, _pane=Str, height=400, loading_indicator=True)
            [3] Divider()